In [1]:
words = open('names_es_ar.txt').read().splitlines()

In [2]:
chars = sorted(list(set("".join(words))))
n = len(chars)
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i: s for s, i in stoi.items()}

In [3]:
import torch
import torch.nn.functional as F


In [4]:
N = torch.zeros((n + 1, n + 1), dtype=torch.int32)

for w in words[:]:
    chs = ["."] + list(w) + ["."]
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1

In [5]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.figure(figsize=(64,64))
# plt.imshow(N, cmap='Blues')
# for i in range(n+1):
#     for j in range(n+1):
#         chstr = itos[i] + itos[j]
#         plt.text(j, i, chstr, ha="center", va="bottom", color="gray")
#         plt.text(j, i, N[i,j].item(), ha="center", va="top", color="gray")
# plt.axis('off')
# plt.show()

In [6]:
N.shape

torch.Size([89, 89])

In [7]:
p = N[0].float()
p = p / p.sum()
p

tensor([0.0000e+00, 1.4629e-03, 0.0000e+00, 1.3351e-01, 5.2003e-02, 4.1291e-02,
        4.7190e-02, 5.6006e-02, 3.5215e-02, 3.9242e-02, 1.0078e-02, 3.4243e-02,
        6.5522e-02, 2.0095e-02, 7.9756e-02, 1.0423e-01, 4.8391e-02, 1.1870e-02,
        1.8327e-02, 8.2439e-04, 2.6547e-02, 5.1808e-02, 4.6156e-02, 8.3249e-03,
        2.0410e-02, 5.5801e-03, 5.1179e-03, 2.1930e-02, 1.4267e-02, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.7652e-06, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        4.7652e-06, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 9.5305e-06, 0.0000e+00, 3.7645e-04,
        4.7652e-06, 0.0000e+00, 2.3826e-05, 4.7652e-05, 0.0000e+00, 0.0000e+00,
        4.7652e-06, 0.0000e+00, 3.8122e-05, 5.7183e-05, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+

In [8]:
p.sum()

tensor(1.)

In [9]:
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
# ix
itos[ix]

'A'

In [10]:
g = torch.Generator().manual_seed(2147483647)
p = torch.rand(3, generator=g)
p = p / p.sum()
p

tensor([0.6064, 0.3033, 0.0903])

In [11]:
torch.multinomial(p, num_samples=100, replacement=True, generator=g)

tensor([1, 1, 2, 0, 0, 2, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 2, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 1, 0,
        0, 1, 1, 1])

In [12]:
# Smoothing
P = (N + 1).float()
P /= P.sum(dim=1, keepdim=True)
P

tensor([[4.7632e-06, 1.4671e-03, 4.7632e-06,  ..., 4.7632e-06, 4.7632e-06,
         4.7632e-06],
        [7.4215e-03, 8.3341e-05, 8.3341e-06,  ..., 4.1670e-06, 4.1670e-06,
         4.1670e-06],
        [9.3458e-03, 2.8037e-02, 9.3458e-03,  ..., 9.3458e-03, 9.3458e-03,
         9.3458e-03],
        ...,
        [2.4194e-02, 3.2258e-02, 8.0645e-03,  ..., 8.0645e-03, 8.0645e-03,
         8.0645e-03],
        [1.1111e-02, 1.1111e-02, 1.1111e-02,  ..., 1.1111e-02, 1.1111e-02,
         1.1111e-02],
        [1.1111e-02, 2.2222e-02, 1.1111e-02,  ..., 1.1111e-02, 1.1111e-02,
         1.1111e-02]])

In [13]:
ix = 0

for i in range(10):
    while True:
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        if ix == 0:
            break
        print(itos[ix], end="")
    print()


Do 
Favis
Jare Al
Dah Va
Ga
Deladivir
My
Ado
Kens Nasto
Coagusinc Aabi


In [14]:
log_likelihood = 0.0
n = 0
for w in words[:]:
    chs = ["."] + list(w) + ["."]
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        log_likelihood += torch.log(P[ix1, ix2])
        n += 1
nll = -log_likelihood
print(f"{log_likelihood=}")
print(f"{nll/n=}")

log_likelihood=tensor(-7390154.5000)
nll/n=tensor(2.3178)


In [15]:
x = 1/89
torch.log(torch.tensor(x))

tensor(-4.4886)